# Naive Bayes

<font size=4>
&emsp;&emsp;In GDA, the feature vectors $x$ were continuous, real-valued vectors. Lets now talk about a different learning algorithm in which the $x$'s are discrete-valued.
</font>

***

## 1. Multi-variate Bernoulli Event Model

<font size=4>
&emsp;&emsp;For our motivating example, consider building an email spam filter using machine learning. Here, we wish to classify messages according to whether they are unsolicited commercial (spam) email, or non-spam email. After learning to do this, we can then have our mail reader automatically filter out the spam messages and perhaps place them in a separate mail folder. Classifying emails is one example of a broader set of problems called **text classification**.
</font>

<font size=4>
&emsp;&emsp;We will **represent an email** via a feature vector whose length is equal to the number of words in the dictionary. Specifically, if an email contains the *$i$*-th word of the dictionary, then we will set $x_i=1$; otherwise, we let $x_i=0$. For instance, the vector
</font>

<font size=4>
$$
x = \begin{bmatrix} 1 \\ 0 \\ 0 \\ \vdots \\ 1 \\ \vdots \\ 0 \end{bmatrix}, \quad
\begin{matrix} a \\ aardvark \\ aardwolf \\ \vdots \\ buy \\ \vdots \\ zygmurgy \end{matrix}
$$
</font>

<font size=4>
&emsp;&emsp;is used to represent an email that contains the words "a" and "buy", but not "aardvark", "aardwolf" or "zygmurgy". The set of words encoded into the feature is called the **vocabulary**, so the dimension of $x$ is equal to the size of the vocabulary.
</font>

<font size=4>
&emsp;&emsp;Actually, rather than looking through an english dictionary for the list of all english words, in practice it is more common to look through our training set and encode in our feature vector only the words that occur at least once there. Apart from reducing the number of words modeled and hence reducing our computational and space requirements, this also has the advantage of allowing us to model/include as a feature many words that may appear in your email (such as "cs229") but that you won't find in dictionary. Sometimes (as in the homework), we also exclude the very high frequency words (which will be words like "the", "of", "and"; these high frequency, "content free" words are called **stop words**) since they occur in so many documents and do little to indicate whether an email is spam or non-spam.
</font>

<font size=4>
&emsp;&emsp;Having chosen our feature vector, we now want to build a discriminative model. So, we have to model $p(x\,|\,y)$. But if we have, say, a vocabulary of 50000 words, then $x\in\{0,\,1\}^{50000}$ ( $x$ is a 50000-dimensional vector of 0's and 1's ), and if we were to model $x$ explicitly with a multinomial distribution over the $2^{50000}$ possible outcomes, then we'd end up with a ($2^{50000}-1$)-dimensional parameter vector. This is clearly too many parameters.
</font> 

<font size=4>
&emsp;&emsp;To model $p(x\,|\,y)$, we will therefore make a very strong assumption. We will assume that the $x_i$'s are conditionally independent given $y$. This assumption is called the **Naive Bayes (NB) assumption**, and the resulting algorithm is called the **Naive Bayes classifier**.
</font>

<font size=4>
&emsp;&emsp;We now have:
</font>

<font size=4>
$$
\begin{align}
p(x_1,\cdots,x_{50000}\,|\,y) 
&= p(x_1\,|\,y)p(x_2\,|\,y,x_1)p(x_3\,|\,y,x_1,x_2){\cdots}p(x_{50000}\,|\,y,x_1,\cdots,x_{49999}) \\
&= p(x_1\,|\,y)p(x_2\,|\,y)p(x_3\,|\,y){\cdots}p(x_{50000}\,|\,y) \\
&= \prod_{i=1}^{n}p(x_i\,|\,y)
\end{align}
$$
</font>

<font size=4>
&emsp;&emsp;The first equality simply follows from the usual properties of probabilities, and the second equality used the NB assumption. We note that even though the Naive Bayes assumption is an extremely strong assumptions, the resulting algorithm works well on many problems.
</font>

<font size=4>
&emsp;&emsp;Our model is parameterized by
</font>

<font size=4>
$$
\begin{align}
\phi_{i\,|\,y=1}&=p(x_i=1\,|\,y=1) \\
\phi_{i\,|\,y=0}&=p(x_i=1\,|\,y=0) \\
\phi_y&=p(y=1)
\end{align}
$$
</font>

<font size=4>
&emsp;&emsp;As usual, given a training set $\{(x^{(i)}, y^{(i)});i=1,\cdots,m\}$, we can write down the joint likelihood of the data:
</font>

<font size=4>
$$
\mathcal{L}(\phi_y,\phi_{i\,|\,y=0},\phi_{i\,|\,y=1}) = \prod_{i=1}^mp(x^{(i)},y^{(i)})
$$
</font>

<font size=4>
&emsp;&emsp;Maximizing this with respect to $\phi_y,\phi_{i\,|\,y=0},\phi_{i\,|\,y=1}$ gives the maximum likelihood estimates:
</font>

<font size=4>
\begin{align}
\phi_{j\,|\,y=1} &= \frac{\sum_{i=1}^{m}1\{x_j^{(i)}=1\,\land\,y^{(i)}=1\}}{\sum_{i=1}^{m}1\{y^{(i)}=1\}} \\
\phi_{j\,|\,y=0} &= \frac{\sum_{i=1}^{m}1\{x_j^{(i)}=1\,\land\,y^{(i)}=0\}}{\sum_{i=1}^{m}1\{y^{(i)}=0\}} \\
\phi_{y} &= \frac{\sum_{i=1}^m1\{y^{(i)}=1\}}{m}
\end{align}
</font>

<font size=4>
&emsp;&emsp;In the equations above, the "$\land$" symbol means "and". The parameters have a very natural interpretation. For instance, $\phi_{j\,|\,y=1}$ is just the fraction of the spam ($y=1$) emails in which word $j$ does appear.
</font>

<font size=4>
&emsp;&emsp;Having fit all these parameters, to make a prediction on a new example with features $x$, we then simply calculate:
</font>

<font size=4>
$$
\begin{align}
p(y=1\,|\,x) 
&= \frac{p(x\,|\,y=1)p(y=1)}{p(x)} \\
&= \frac{(\prod^n_{i=1}p(x_i\,|\,y=1))\,p(y=1)}{(\prod^n_{i=1}p(x_i\,|\,y=1))\,p(y=1)+(\prod^n_{i=1}p(x_i\,|\,y=0))\,p(y=0)}
\end{align}
$$
</font>

<font size=4>
and pick whichever class has the higher posterior probability.
</font>

***

### 详细推导：

<font size=4>
&emsp;&emsp;已知
</font>

<font size=4>
$$
\begin{align}
y\,&\sim\,Bernoulli(\phi_y) \\
x_j\,|\,y=0\,&\sim\,Bernoulli(\phi_{j\,|\,y=0}) \\
x_j\,|\,y=1\,&\sim\,Bernoulli(\phi_{j\,|\,y=1})
\end{align}
$$
</font>

<font size=4>
&emsp;&emsp;对数似然函数
</font>

<font size=4>
$$
\begin{align}
l(\phi_y,\phi_{j\,|\,y=0},\phi_{j\,|\,y=1})
&= log\prod_{i=1}^mp(x^{(i)},y^{(i)}) \\
&= log\prod_{i=1}^mp(x^{(i)}\,|\,y^{(i)})p(y^{(i)}) \\
&= \sum_{i=1}^{m}logp(x^{(i)}\,|\,y^{(i)})+\sum_{i=1}^mlog\,p(y^{(i)}) \\
&= \sum_{i=1}^mlog\prod^{n}_{j=1}p(x^{(i)}_j\,|\,y^{(i)}) + \sum_{i=1}^mlog\,p(y^{(i)}) \\
&= \sum_{i=1}^mlog\prod^{n}_{j=1}p(x^{(i)}_j\,|\,y^{(i)}=0)^{1-y^{(i)}}\,{\cdot}\,p(x^{(i)}_j\,|\,y^{(i)}=1)^{y^{(i)}} + \sum_{i=1}^mlog\,p(y^{(i)}) \\
&= \sum_{i=1}^m\sum^n_{j=1}\left( (1-y^{(i)})\,log\,p(x^{(i)}_j\,|\,y^{(i)}=0)+y^{(i)}\,log\,p(x^{(i)}_j\,|\,y^{(i)}=1) \right)+ \sum_{i=1}^mlog\,p(y^{(i)}) \\
&= \sum_{i=1}^m\sum^n_{j=1}(1-y^{(i)})\,log\,p(x^{(i)}_j\,|\,y^{(i)}=0)+\sum_{i=1}^m\sum^n_{j=1}y^{(i)}\,log\,p(x^{(i)}_j\,|\,y^{(i)}=1) + \sum_{i=1}^mlog\,p(y^{(i)})
\end{align}
$$
</font>

<font size=4>
&emsp;&emsp;注意，此函数分为三个部分，其中，$\phi_{j\,|\,y=0}$ 只与第一部分有关，$\phi_{j\,|\,y=1}$ 只与第二部分有关，$\phi_y$ 只与第三部分有关。
</font>

<font size=4>
首先，求 $\phi_{j\,|\,y=0}$
</font>

<font size=4>
\begin{align}
\triangledown_{\phi_{j\,|\,y=0}}l(\phi_y,\phi_{j\,|\,y=0},\phi_{j\,|\,y=1})
&= \triangledown_{\phi_{j\,|\,y=0}}\sum^m_{i=1}\sum^n_{j=1}(1-y^{(i)})\,log\,p(x^{(i)}_j\,|\,y^{(i)}=0) \\
&= \triangledown_{\phi_{j\,|\,y=0}}\sum^m_{i=1}\sum^n_{j=1}(1-y^{(i)})\,log\,\phi_{j\,|\,y=0}^{x_j^{(i)}}\,\cdot\,(1-\phi_{j\,|\,y=0})^{(1-x_j^{(i)})} \\
&= \triangledown_{\phi_{j\,|\,y=0}}\sum^m_{i=1}\sum^n_{j=1}1\{y^{(i)}=0\}\left( x_j^{(i)}\,log\,\phi_{j\,|\,y=0}+(1-x_j^{(i)})\,log\,(1-\phi_{j\,|\,y=0}) \right) \\
&= \sum^m_{i=1}1\{y^{(i)}=0\}\left( x_j^{(i)}\frac{1}{\phi_{j\,|\,y=0}}-(1-x_j^{(i)})\frac{1}{1-\phi_{j\,|\,y=0}} \right) \\
&= \frac{\sum^m_{i=1}1\{y^{(i)}=0\}x_j^{(i)}}{\phi_{j\,|\,y=0}}-\frac{\sum^m_{i=1}1\{y^{(i)}=0\}(1-x_j^{(i)})}{1-\phi_{j\,|\,y=0}}
\end{align}
</font>

<font size=4>
&emsp;&emsp;令其为零，即
</font>

<font size=4>
$$
\begin{align}
\frac{\sum^m_{i=1}1\{y^{(i)}=0\}x_j^{(i)}}{\phi_{j\,|\,y=0}} &= \frac{\sum^m_{i=1}1\{y^{(i)}=0\}(1-x_j^{(i)})}{1-\phi_{j\,|\,y=0}} \\
\sum^m_{i=1}1\{y^{(i)}=0\}x_j^{(i)}-\phi_{j\,|\,y=0}\sum^m_{i=1}1\{y^{(i)}=0\}x_j^{(i)} &= \phi_{j\,|\,y=0}\sum^m_{i=1}1\{y^{(i)}=0\}(1-x_j^{(i)}) \\
\sum^m_{i=1}1\{y^{(i)}=0\}1\{x_j^{(i)}=1\} &= \phi_{j\,|\,y=0}\sum^m_{i=1}1\{y^{(i)}=0\} \\
\phi_{j\,|\,y=0} &= \frac{\sum^m_{i=1}1\{x_j^{(i)}=1\,{\land}\,y^{(i)}=0\}}{\sum^m_{i=1}1\{y^{(i)}=0\}}
\end{align}
$$
</font>

<font size=4>
&emsp;&emsp;$\phi_{j\,|\,y=1}$，同理可得。
</font>

<font size=4>
最后，求 $\phi_y$
</font>

<font size=4>
$$
\begin{align}
\triangledown{\phi_y}l(\phi_y,\phi_{j\,|\,y=0},\phi_{j\,|\,y=1})
&= \triangledown{\phi_y}\sum^{m}_{i=1}log\,p(y^{(i)}) \\
&= \triangledown{\phi_y}\sum^{m}_{i=1}log\,\phi_y^{y^{(i)}}(1-\phi_y)^{(1-y^{(i)})} \\
&= \triangledown{\phi_y}\sum^{m}_{i=1}\left( y^{(i)}log\phi_y+(1-y^{(i)})log(1-\phi_y) \right) \\
&= \sum^{m}_{i=1}y^{(i)}\frac{1}{\phi_y}-\sum^{m}_{i=1}(1-y^{(i)})\frac{1}{1-\phi_y}
\end{align}
$$
</font>

<font size=4>
&emsp;&emsp;令其为零，可得
</font>

<font size=4>
$$
\begin{align}
\sum^{m}_{i=1}y^{(i)}\frac{1}{\phi_y} &= \sum^{m}_{i=1}(1-y^{(i)})\frac{1}{1-\phi_y} \\
\sum^m_{i=1}y^{(i)}-\phi_y\sum^m_{i=1}y^{(i)} &= \phi_y\sum^m_{i=1}(1-y^{(i)}) \\
\sum^m_{i=1}1\{y^{(i)}=1\}-\phi_y\sum^m_{i=1}1\{y^{(i)}=1\} &= \phi_y\sum^m_{i=1}1\{y^{(i)}=0\} \\
\sum^m_{i=1}1\{y^{(i)}=1\} &= m\phi_y \\
\phi_y &= \frac{\sum^m_{i=1}1\{y^{(i)}=1\}}{m}
\end{align}
$$
</font>

***

<font size=4>
&emsp;&emsp;Lastly, we note that while we have developed the Naive Bayes algorithm mainly for the case of problems where the features $x_i$ are binary-valued, the generalization to where $x_i$ can take values in $\{1,2,\cdots,k\}$ is straightforward. Here, we would simply model $p(x_i\,|\,y)$ as multinomial rather than as Bernoulli. Indeed, even if some original inpute attribute (say, the living area of a house, as in our earlier example) were continuous valued, it is quite common to **discretize** it——that is turn it into a small set of discrete values——and apply Naive Bayes. 
</font>

<font size=4>
&emsp;&emsp;When the original, continuous-valued attributes are well-modeled by a multivariate normal distribution, discretizing the features and using Naive Bayes (instead of GDA) will often result in a better classifier.
</font>

In [ ]:
<font size=4>
&emsp;&emsp;
</font>